In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()

filter = np.where((train_labels ==6) | (train_labels == 7) | (train_labels == 8))
train_images = train_images[filter[0]].astype('float32')
train_images = (train_images - 127.5) / 127.5


In [ ]:
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3,3), padding='same', input_shape=(32,32,3)),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2D(128, (3,3), strides=(2,2), padding='same'),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2D(128, (3,3), strides=(2,2), padding='same'),
        layers.LeakyReLU(alpha=0.2),

        # Added extra layers for more capacity
        layers.Conv2D(256, (3,3), strides=(2,2), padding='same'),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2D(512, (3,3), strides=(2,2), padding='same'),
        layers.LeakyReLU(alpha=0.2),

        layers.Conv2D(512, (3,3), padding='same'),
        layers.LeakyReLU(alpha=0.2),

        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid'),
    ])
    return model



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def build_generator():
    model = tf.keras.Sequential([
        # Input projection and reshape
        layers.Dense(512*4*4, input_dim=100),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((4,4,512)),

        # First upsampling block (4x4 → 8x8)
        layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        # Additional conv layer at same resolution
        layers.Conv2D(256, (3,3), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        # Second upsampling block (8x8 → 16x16)
        layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        # Additional conv layer at same resolution
        layers.Conv2D(128, (3,3), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        # Third upsampling block (16x16 → 32x32)
        layers.Conv2DTranspose(64, (4,4), strides=(2,2), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        # Additional refinement layers at final resolution
        layers.Conv2D(64, (3,3), padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(alpha=0.2),

        # Final output layer
        layers.Conv2D(3, (3,3), activation='tanh', padding='same'),
    ])
    return model



In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + \
                    cross_entropy(tf.zeros_like(fake_output), fake_output)

    gradients_of_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc, discriminator.trainable_variables))

    return gen_loss, disc_loss

def generate_images(epoch):
    predictions = generator(tf.random.normal([16, 100]), training=False)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i] * 127.5 + 127.5).numpy().astype('uint8'))
        plt.axis('off')
    plt.show()
    plt.close()


In [ ]:
EPOCHS = 1000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images) \
    .shuffle(6000).batch(BATCH_SIZE)

generator = build_generator()
discriminator = build_discriminator()

for epoch in range(EPOCHS):
    for image_batch in train_dataset:
        gen_loss, disc_loss = train_step(image_batch)

    generate_images(epoch)
    print(f'Epoch {epoch+1} | Gen loss: {gen_loss:.4f} | Disc loss: {disc_loss:.4f}')


In [ ]:
for epoch in range(900):
    for image_batch in train_dataset:
        gen_loss, disc_loss = train_step(image_batch)

    generate_images(epoch)
    print(f'Epoch {epoch+1} | Gen loss: {gen_loss:.4f} | Disc loss: {disc_loss:.4f}')


In [ ]:
generator.save('generator_cifar10_1000epochs.keras')
discriminator.save('discriminator_cifar10_1000epochs.keras')